## WDPA Attribution

In [12]:
from tqdm.auto import tqdm

In [13]:
input_ranges = "Global_Grid"
# World Database of Protected Areas
wdpa_point = "https://services5.arcgis.com/Mj0hjvkNtV7NRhA7/arcgis/rest/services/WDPA_v0/FeatureServer/0"
wdpa_poly = "https://services5.arcgis.com/Mj0hjvkNtV7NRhA7/arcgis/rest/services/WDPA_v0/FeatureServer/1"

In [10]:
# copy and project input datasets
with arcpy.EnvManager(outputCoordinateSystem=input_ranges):
    arcpy.management.CopyFeatures(
        in_features=wdpa_point,
        out_feature_class="WDPA_point_Latest"
    )

    arcpy.management.CopyFeatures(
        in_features="WDPA_poly_Latest",
        out_feature_class="WDPA_poly_Latest"
    )

In [11]:
# create separate layers for Ramsar sites
arcpy.management.MakeFeatureLayer(
    in_features="WDPA_poly_Latest",
    out_layer="Ramsar_poly",
    where_clause="desig_eng LIKE '%Ramsar%' Or desig_eng LIKE '%RAMSAR%'",
    workspace=None,
    field_info="OBJECTID OBJECTID VISIBLE NONE;Shape Shape VISIBLE NONE;wdpaid wdpaid VISIBLE NONE;wdpa_pid wdpa_pid VISIBLE NONE;pa_def pa_def VISIBLE NONE;name name VISIBLE NONE;orig_name orig_name VISIBLE NONE;desig desig VISIBLE NONE;desig_eng desig_eng VISIBLE NONE;desig_type desig_type VISIBLE NONE;iucn_cat iucn_cat VISIBLE NONE;int_crit int_crit VISIBLE NONE;marine marine VISIBLE NONE;rep_m_area rep_m_area VISIBLE NONE;gis_m_area gis_m_area VISIBLE NONE;rep_area rep_area VISIBLE NONE;gis_area gis_area VISIBLE NONE;no_take no_take VISIBLE NONE;no_tk_area no_tk_area VISIBLE NONE;status status VISIBLE NONE;status_yr status_yr VISIBLE NONE;gov_type gov_type VISIBLE NONE;own_type own_type VISIBLE NONE;mang_auth mang_auth VISIBLE NONE;mang_plan mang_plan VISIBLE NONE;verif verif VISIBLE NONE;metadataid metadataid VISIBLE NONE;sub_loc sub_loc VISIBLE NONE;parent_iso3 parent_iso3 VISIBLE NONE;iso3 iso3 VISIBLE NONE;Shape_Length Shape_Length VISIBLE NONE;Shape_Area Shape_Area VISIBLE NONE"
)

arcpy.management.MakeFeatureLayer(
    in_features="WDPA_point_Latest",
    out_layer="Ramsar_point",
    where_clause="desig_eng LIKE '%Ramsar%' Or desig_eng LIKE '%RAMSAR%'",
    workspace=None,
    field_info="OBJECTID OBJECTID VISIBLE NONE;Shape Shape VISIBLE NONE;wdpaid wdpaid VISIBLE NONE;wdpa_pid wdpa_pid VISIBLE NONE;pa_def pa_def VISIBLE NONE;name name VISIBLE NONE;orig_name orig_name VISIBLE NONE;desig desig VISIBLE NONE;desig_eng desig_eng VISIBLE NONE;desig_type desig_type VISIBLE NONE;iucn_cat iucn_cat VISIBLE NONE;int_crit int_crit VISIBLE NONE;marine marine VISIBLE NONE;rep_m_area rep_m_area VISIBLE NONE;gis_m_area gis_m_area VISIBLE NONE;rep_area rep_area VISIBLE NONE;gis_area gis_area VISIBLE NONE;no_take no_take VISIBLE NONE;no_tk_area no_tk_area VISIBLE NONE;status status VISIBLE NONE;status_yr status_yr VISIBLE NONE;gov_type gov_type VISIBLE NONE;own_type own_type VISIBLE NONE;mang_auth mang_auth VISIBLE NONE;mang_plan mang_plan VISIBLE NONE;verif verif VISIBLE NONE;metadataid metadataid VISIBLE NONE;sub_loc sub_loc VISIBLE NONE;parent_iso3 parent_iso3 VISIBLE NONE;iso3 iso3 VISIBLE NONE;Shape_Length Shape_Length VISIBLE NONE;Shape_Area Shape_Area VISIBLE NONE"
)

<Result 'Ramsar_point'>

In [12]:
# get count of Ramsar sites
arcpy.analysis.SummarizeWithin(
    in_polygons=input_ranges,
    in_sum_features="Ramsar_point",
    out_feature_class=f"{input_ranges}_WDPA_0"
)

arcpy.analysis.SummarizeWithin(
    in_polygons=f"{input_ranges}_WDPA_0",
    in_sum_features="Ramsar_poly",
    out_feature_class=f"{input_ranges}_WDPA_1"
)

arcpy.management.CalculateFields(
    in_table=f"{input_ranges}_WDPA_1",
    expression_type="PYTHON3",
    fields="Ramsar_count '!Point_Count! + !Polygon_Count!' # SHORT"
)

arcpy.management.JoinField(
    in_data=input_ranges,
    in_field="sci_name",
    join_table=f"{input_ranges}_WDPA_1",
    join_field="sci_name",
    fields="Ramsar_count"
)


<Result 'Global_Grid'>

In [8]:
## get total protected area overlapping with each range
# create layer with established or registered protected areas
arcpy.management.MakeFeatureLayer(
    in_features="WDPA_poly_Latest",
    out_layer="Present_PAs",
    where_clause="status <> 'Proposed'"
)

<Result 'Present_PAs'>

In [10]:
# dissolve to avoid double counting for areas with multiple overlapping designations
arcpy.analysis.PairwiseDissolve(
    in_features="Present_PAs",
    out_feature_class="WDPA_poly_dissolved",
    multi_part="SINGLE_PART"
)

<class 'arcgisscripting.ExecuteError'>: ERROR 160385: Workspace or data source is read only.
Failed to execute (PairwiseDissolve).


In [14]:
# attribute IUCN ranges with area-weighted means based on proportional overlaps with non-null country values

fields = ['SHAPE@','Protected_Percent']

arcpy.management.AddFields(input_ranges,"Protected_Percent FLOAT")

with arcpy.da.UpdateCursor(input_ranges,fields) as cursor:
    
    for row in tqdm(cursor,total=int(arcpy.management.GetCount(input_ranges)[0])):
        
        arcpy.analysis.PairwiseClip(
            in_features="WDPA_poly_dissolved",
            clip_features=row[0],
            out_feature_class="currAreas"
        )

        row[1] = sum([c[0] for c in arcpy.da.SearchCursor("currAreas","Shape_Area")]) / row[0].area * 100
        
        cursor.updateRow(row)

  0%|          | 0/16528 [00:00<?, ?it/s]


<class 'arcgisscripting.ExecuteError'>: Failed to execute. Parameters are not valid.
ERROR 000732: Input Features: Dataset WDPA_poly_dissolved does not exist or is not supported
Failed to execute (PairwiseClip).


In [ ]:
# export table

arcpy.conversion.ExportTable(
    in_table=input_ranges,
    out_table=r"R:FWL\Arismendi-Lab\Andres\Gilbert_Freshwater_Fish_Analysis\Revised_Analysis_NatureCommunications\Input_datasets\Tabular_data_by_species\WDPA_Attributes.csv",
    where_clause="",
    use_field_alias_as_name="NOT_USE_ALIAS",
    field_mapping=f'sci_name "sci_name" true true false 100 Text 0 0,First,#,{input_ranges},sci_name,0,99;Protected_Percent "Protected_Percent" true true false 4 Float 0 0,First,#,{input_ranges},Protected_Percent,-1,-1;Ramsar_count "Ramsar_count" true true false 2 Short 0 0,First,#,{input_ranges},Ramsar_count,-1,-1',
    sort_field=None
)

In [ ]:
# delete intermediate outputs
arcpy.management.Delete(
    in_data=fr"Ramsar_point;Ramsar_poly;Present_PAs;WDPA_point_Latest;WDPA_poly_Latest;WDPA_poly_dissolved;{arcpy.env.workspace}\Extant_Native_Ranges_WDPA_0;{arcpy.env.workspace}\Extant_Native_Ranges_WDPA_1;{arcpy.env.workspace}\WDPA_point_Latest;{arcpy.env.workspace}\WDPA_poly_dissolved;{arcpy.env.workspace}\WDPA_poly_Latest;{arcpy.env.workspace}\currAreas",
)